### 🧾 Final SQL Query (Full Structure)
```sql
SELECT 
    CASE WHEN i.BIN = '      ' THEN 'BLANK_BIN' ELSE i.BIN END AS BIN,

    CASE 
        WHEN i.BIN = 'CASOUT' THEN 'Any where on 4300 Floor'
        WHEN i.BIN = 'CNL' THEN 'Any floor in any building'
        WHEN i.BIN = 'L1STAG' THEN 'Logistics Room 1 - Check Floor'
        WHEN i.BIN = 'L2STAG' THEN 'Logistics Room 2 - Check Floor'
        WHEN i.BIN = 'L3STAG' THEN 'Logistics Room 3 - Check Floor'
        WHEN i.BIN = 'L4STAG' THEN 'Logistics Room 4 - Check Floor'
        WHEN i.BIN = 'MACOUT' THEN 'Any where on Machine Shop Floor'
        ELSE SUBSTRING(i.BIN, 1, 2)
    END AS BUILDING,

    CASE 
        WHEN i.BIN IN ('CASOUT', 'CNL', 'L1STAG', 'L2STAG', 'L3STAG', 'L4STAG', 'MASOUT') THEN NULL
        ELSE SUBSTRING(i.BIN, 3, 2)
    END AS BAY_SHELF,

    CASE 
        WHEN i.BIN IN ('CASOUT', 'CNL', 'L1STAG', 'L2STAG', 'L3STAG', 'L4STAG', 'MASOUT') THEN NULL
        ELSE SUBSTRING(i.BIN, 5, 2)
    END AS SHELF,

    CASE 
        WHEN i.BIN LIKE 'L1%' THEN 'Logistics Room 1'
        WHEN i.BIN LIKE 'L2%' THEN 'Logistics Room 2'
        WHEN i.BIN LIKE 'L3%' THEN 'Logistics Room 3'
        WHEN i.BIN LIKE 'L4%' THEN 'Logistics Room 4'
        WHEN i.BIN LIKE 'W1%' THEN 'Warehouse 1'
        WHEN i.BIN LIKE 'W2%' THEN 'Warehouse 2'
        WHEN i.BIN = 'WAREH1' THEN 'Warehouse 1'
        WHEN i.BIN = 'WAREH3' THEN 'Warehouse 3'
        WHEN i.BIN = 'WAREH4' THEN 'Warehouse 4'
        WHEN i.BIN LIKE 'M1%' THEN 'Machine Shop Room 1'
        WHEN i.BIN LIKE 'M2%' THEN 'Machine Shop Room 2'
        WHEN i.BIN LIKE 'M3%' THEN 'Machine Shop Room 3'
        WHEN i.BIN LIKE 'M4%' THEN 'Machine Shop Room 4'
        WHEN i.BIN LIKE 'MAC%' THEN 'Machine Shop'
        WHEN i.BIN = 'CASOUT' THEN 'Casting'
        WHEN i.BIN = 'CNL' THEN 'CNL'
        ELSE 'Other'
    END AS BUILD_FIELD,

    v.PRODUCT_LINE,
    i.PART,
    i.QUANTITY,
    pl.PRODUCT_LINE_DESC,
    pl.PRODUCT_LINE_NAME

FROM 
    v_item_master i
JOIN 
    v_inventory_mstr v ON i.PART = v.PART
LEFT JOIN 
    V_PRODUCT_LINE pl ON v.PRODUCT_LINE = pl.PRODUCT_LINE

WHERE 
    i.BIN IS NOT NULL
    AND v.PRODUCT_LINE != 'MP'
    AND i.BIN NOT IN ('HOLD', '4300MP', 'K')
    AND pl.PRODUCT_LINE NOT IN (
        'ZZ', 'ZL', 'WP', 'XD', 'XR', 'XS', 'XX', 'ZA', 'ZC', 'ZD', 'ZE',
        'ZH', 'ZI', 'ZP', 'ZR', 'ZS', 'ZV', 'ZW', 'CS',
        '12', '14', '15', '16', '17', '18', '9L', '9P', 'IM', 'IP'
    )
    AND i.QUANTITY > 0

ORDER BY 4, 1;
```

## 📖 SQL Query Explanation

This section provides a full breakdown of the SQL used to generate the Physical Inventory report.

---

### 🔹 BIN Normalization
```sql
CASE WHEN i.BIN = '      ' THEN 'BLANK_BIN' ELSE i.BIN END AS BIN
```
Empty BINs (which often appear as six spaces) are replaced with 'BLANK_BIN' for clarity.

---

### 🔹 BUILDING Logic
```sql
CASE 
  WHEN i.BIN = 'CASOUT' THEN 'Any where on 4300 Floor'
  ...
  ELSE SUBSTRING(i.BIN, 1, 2)
END AS BUILDING
```
Special BINs are hardcoded to custom locations. Others extract the first two characters for building code.

---

### 🔹 BAY_SHELF and SHELF Parsing
```sql
SUBSTRING(i.BIN, 3, 2) AS BAY_SHELF
SUBSTRING(i.BIN, 5, 2) AS SHELF
```
These slice the BIN string to extract shelf identifiers. Special BINs skip this and return `NULL`.

---

### 🔹 BUILD_FIELD (Grouping Label)
```sql
CASE 
  WHEN i.BIN LIKE 'L1%' THEN 'Logistics Room 1'
  ...
  ELSE 'Other'
END AS BUILD_FIELD
```
Provides user-friendly descriptions used for page-level grouping in SSRS reports.

---

### 🔹 Data Joins
```sql
FROM v_item_master i
JOIN v_inventory_mstr v ON i.PART = v.PART
LEFT JOIN V_PRODUCT_LINE pl ON v.PRODUCT_LINE = pl.PRODUCT_LINE
```
Brings in:
- BIN and inventory data (`v_item_master`)
- Part-level classification (`v_inventory_mstr`)
- Descriptions for product line codes (`V_PRODUCT_LINE`)

---

### 🔹 Filtering Conditions
```sql
WHERE 
  i.BIN IS NOT NULL
  AND v.PRODUCT_LINE != 'MP'
  AND i.BIN NOT IN ('HOLD', '4300MP', 'K')
  AND pl.PRODUCT_LINE NOT IN ('ZZ', ..., 'IP')
  AND i.QUANTITY > 0
```
Excludes:
- Blank BINs
- Internal-only or test parts
- Zero-quantity inventory
- Obsolete or irrelevant product lines

---

### 🔹 Final Sorting
```sql
ORDER BY 4, 1;
```
Ensures data is presented by `PART` (column 4) and then by BIN.

---

This logic significantly **reduces manual prep time**, improves consistency, and allows **scalable reporting** across tens of thousands of SKUs.
